### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [ ]:
from modelModule.model4 import VAE4
import torch

In [ ]:
model = VAE4(dim=57, nhead=3)
a = torch.randn((10,57))
b = torch.randn((10,57))
x, y, z = model(a,b)

In [ ]:
x

### dataloader验证

In [11]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [12]:
dataset = FlatDataset('Credit_Card无标签.csv')
print(dataset.Max_Val.shape)
print(dataset.Min_Val)

(23,)
[ 1.00000e+04  1.00000e+00  0.00000e+00  0.00000e+00  2.10000e+01
 -2.00000e+00 -2.00000e+00 -2.00000e+00 -2.00000e+00 -2.00000e+00
 -2.00000e+00 -1.65580e+05 -6.97770e+04 -1.57264e+05 -1.70000e+05
 -8.13340e+04 -3.39603e+05  0.00000e+00  0.00000e+00  0.00000e+00
  0.00000e+00  0.00000e+00  0.00000e+00]


In [ ]:
dataset = FlatDataset('Letter_train.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
test_data = imputed_data * global_max + global_min # 广播效应

In [ ]:
a = torch.randn(2,3,4)
b = torch.nn.AdaptiveMaxPool1d(output_size=1)(a).squeeze(-1)

In [ ]:
b.shape

### 模型精度测定

In [ ]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
# from modelModule.model2 import VAE2
from modelModule.model4 import VAE4
from torch.utils.data import DataLoader
from utils import result_show, get_missing
import torch
import numpy as np

In [15]:
## 模型参数载入
model = VAE4(dim=23) # nhead=3
checkpoint_path = r'模型参数保存/Credit_model4_norm_way_mean_norm_/version_1/checkpoints/epoch=6-step=826.ckpt'    
data_norm = 'mean_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Credit_Card无标签.csv'
test_csv = r'Credit_Card无标签.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm)
model.get_global_min_max(global_max=train_dataset.Max_Val, global_min=train_dataset.Min_Val)
print('载入成功')

载入成功


In [16]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm)
data_size = 10
src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = src_data.copy()
input[miss_matrix == 0] = np.nan

In [17]:
imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [22]:
for i in range(23):
    print(src_data[0][i], '————', imputed_data[0][i])

20000.0 ———— 126865.17003880395
2.0 ———— 1.755053794793843
2.0 ———— 1.6500473097192514
1.0 ———— 1.4383308989009898
24.0 ———— 45.94341831388587
2.0 ———— 0.5931300952329058
2.0 ———— -1.0287416751599014
-1.0 ———— -1.5984989083934789
-1.0 ———— 1.0639882734490103
-2.0 ———— -0.8592509720597454
-2.0 ———— -0.3567344512909735
3913.0 ———— 33163.04237482895
3102.0 ———— 48470.622956304236
689.0 ———— 43994.160031049694
0.0 ———— 17070.06756781765
0.0 ———— 96726.93283550581
0.0 ———— 19522.191874553475
0.0 ———— 17864.670064013073
689.0 ———— 28926.6563172675
0.0 ———— 20180.1343784211
0.0 ———— 9221.400161149462
0.0 ———— -9560.830164657313
0.0 ———— 10657.446655960894


In [23]:
imputed_data[2]

array([ 1.27094385e+05,  1.75347908e+00,  1.65156672e+00,  1.43909255e+00,
        4.59394976e+01,  5.93566018e-01, -1.02350659e+00, -1.60050535e+00,
        1.06496038e+00, -8.60827675e-01, -3.57075721e-01,  3.35757456e+04,
        4.85894810e+04,  4.38497767e+04,  1.70919797e+04,  9.65101938e+04,
        1.95724033e+04,  1.78935505e+04,  2.88717702e+04,  2.01995417e+04,
        9.24204373e+03, -9.54241788e+03,  1.06294403e+04])

In [24]:
imputed_data[1]

array([ 1.26821086e+05,  1.75615644e+00,  1.64867923e+00,  1.43771208e+00,
        4.59770857e+01,  5.89577602e-01, -1.03447811e+00, -1.60165489e+00,
        1.06606823e+00, -8.57189857e-01, -3.59373143e-01,  3.30047916e+04,
        4.88628168e+04,  4.42414489e+04,  1.69216198e+04,  9.67105065e+04,
        1.96443599e+04,  1.78409785e+04,  2.89965295e+04,  2.02464747e+04,
        9.22752376e+03, -9.55600649e+03,  1.06123797e+04])

In [ ]:
result_show(src_data, imputed_data, miss_matrix)

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))